In [1]:
from langchain_groq.chat_models import ChatGroq
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatGroq(
    model="llama-3.3-70b-versatile",   # or another Groq model
    temperature=0.7,
)

In [20]:
model1  = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3
)


In [21]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    evaluation_score: str

In [22]:
def create_outline(state: BlogState) -> BlogState:

    title = state['title']

    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    state['outline'] = outline

    return state

In [23]:
def create_blog(state: BlogState)->BlogState:

    title = state['title']
    outline = state['outline']


    prompt = f'Write a detailed blog on the title - {title} using the following outling \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [24]:
def eval_blog(state:BlogState)->BlogState:

    outline = state['outline']
    content = state['content']    

    prompt = f"""
You are an expert writing evaluator. Evaluate the following blog based on:

- Structure & flow
- Depth & clarity
- Creativity
- Alignment with outline
- Grammar & readability

Return a score from 1–10 and a short explanation.

---

Outline:
{outline}

---

Blog:
{content}
"""
    evaluation = model1.invoke(prompt).content
    state["evaluation_score"] = evaluation
    return state

    

In [25]:
graph = StateGraph(BlogState)

graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('eval_blog',eval_blog)

graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','eval_blog')
graph.add_edge('eval_blog',END)

workflow = graph.compile()

In [26]:
initial_state = {'title':'Rise of AI in India'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': 'Here is a detailed outline for a blog on the topic "Rise of AI in India":\n\n**I. Introduction**\n\n* Brief overview of Artificial Intelligence (AI) and its increasing importance globally\n* Context of AI in India: growing economy, large talent pool, and government support\n* Thesis statement: India is poised to become a significant player in the global AI landscape, driven by government initiatives, innovative startups, and a large pool of skilled professionals.\n\n**II. History and Evolution of AI in India**\n\n* Early beginnings: Introduction to AI in India, key milestones, and pioneers in the field\n* Government initiatives: National IT Policy (2012), Digital India (2015), and the AI Task Force (2017)\n* Growth of AI research and development: Academic institutions, research centers, and industry collaborations\n\n**III. Current State of AI in India**\n\n* AI adoption: Industries leveraging AI, such as healthcare, finance, and manufacturi